# 工业级散列表

## 什么是好的散列表
1. 散列函数的设计不能太复杂。
    - 过于复杂的散列函数，势必会消耗很多计算时间，也就间接地影响到散列表的性能。
2. 散列函数生成的值要尽可能随机并且均匀分布
    - 这样才能避免或者最小化散列冲突，而且即便出现冲突，散列到每个槽里的数据也会比较平均，不会出现某个槽内数据特别多的情况。

实际工作中，还需要综合考虑各种因素。这些因素有：
- 关键字的长度、特点、分布、还有散列表的大小等

## 装载因子过大怎么办
- 对于没有频繁插入和删除的静态数据集合来说，很容易根据数据的特点、分布等，设计出完美的、极少冲突的散列函数，因为毕竟之前数据都是已知的。
- 对于动态散列表来说，数据集合是频繁变动的，事先无法预估将要加入的数据个数，所以也无法事先申请一个足够大的散列表。随着数据慢慢加入，装载因子就会慢慢变大。当装载因子大到一定程度之后，散列冲突就会变得不可接受

当散列表的装载因子超过某个阈值时，就需要进行扩容。装载因子阈值需要选择得当。如果太大，会导致冲突过多；如果太小，会导致内存浪费严重。装载因子阈值的设置要权衡时间、空间复杂度。  

如果内存空间不紧张，对执行效率要求很高，可以降低负载因子的阈值；相反，如果内存空间紧张，对执行效率要求又不高，可以增加负载因子的值，甚至可以大于 1。

## 如何避免低效扩容
- 不一次性扩容，而是分批次慢慢搬移数据

## 如何选择冲突解决方法
- 开放寻址法
    - 当数据量小，装载因子小的时候，适合采用开放寻址法。
- 链表法
    - 基于链表的散列冲突处理方法比较适合存储大对象、大数据量的散列表
    - 比起开放寻址法，它更加灵活，支持更多的优化策略，比如用红黑树代替链表。

## 工业级散列表举例分析
1. 设计一个散列表，初始默认大小是16。
2. 最大装填因子是0.75，超过就进行扩容，扩容为原先的两倍。
3. 使用链表发解决冲突，当链表长度超过8时将链表转换为红黑树

In [ ]:
class HashNode:
    def __init__(self, key, value, next=None):
        self.key = key
        self.value = value
        self.next = next


class HashMap:
    def __init__(self, size=16):
        self.size = size
        self.count = 0
        self.buckets = [None] * self.size

    def hash(self, key):
        return hash(key) % self.size

    def put(self, key, value):
        index = self.hash(key)
        node = self.buckets[index]
        if node is None:
            self.buckets[index] = HashNode(key, value)
            self.count += 1
            if self.count / self.size > 0.75:
                self._resize()
            return
        prev = None
        while node is not None and node.key != key:
            prev = node
            node = node.next
        if node is not None:
            node.value = value
        else:
            prev.next = HashNode(key, value)
            self.count += 1
            if self.count / self.size > 0.75:
                self._resize()

    def get(self, key):
        index = self.hash(key)
        node = self.buckets[index]
        while node is not None and node.key != key:
            node = node.next
        if node is None:
            return None
        else:
            return node.value

    def _resize(self):
        old_buckets = self.buckets
        self.size *= 2
        self.buckets = [None] * self.size
        self.count = 0
        for node in old_buckets:
            while node is not None:
                self.put(node.key, node.value)
                node = node.next

## 总结
何为一个工业级的散列表？工业级的散列表应该具有哪些特性？
- 支持快速地查询、插入、删除操作；
- 内存占用合理，不能浪费过多的内存空间；
- 性能稳定，极端情况下，散列表的性能也不会退化到无法接受的情况。

如何实现这样一个散列表呢？
- 设计一个合适的散列函数；
- 定义装载因子阈值，并且设计动态扩容策略；
- 选择合适的散列冲突解决方法。